In [1]:
#import packages
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
#read data
data=pd.read_csv('iris_data.data', names=['sepal_length','sepal_width','petal_length','petal_width','species'])

In [3]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = data['species']

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
data['species'] = list(onehot_encoded)

In [5]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,"[1.0, 0.0, 0.0]"
1,4.9,3.0,1.4,0.2,"[1.0, 0.0, 0.0]"
2,4.7,3.2,1.3,0.2,"[1.0, 0.0, 0.0]"
3,4.6,3.1,1.5,0.2,"[1.0, 0.0, 0.0]"
4,5.0,3.6,1.4,0.2,"[1.0, 0.0, 0.0]"


In [6]:
#splitting the data into training and test set
from sklearn.model_selection import train_test_split
X = data.drop(['species'],axis = 1)
y = data['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state = 2018)

In [7]:
# Parameters
learning_rate = 0.1
epoch = 500

# Network Parameters
num_input = 4 
num_classes = 3 

# tf Graph input
X = tf.placeholder("float", [None, num_input])
y = tf.placeholder("float", [None, num_classes])

In [8]:
# Store weight & bias
weights = tf.Variable(tf.random_normal([num_input, num_classes]))
biases = tf.Variable(tf.random_normal([num_classes]))

In [9]:
# Create model
def neural_net(x):
    out_layer = tf.matmul(x, weights) + biases
    return out_layer

In [10]:
# Construct model
out_layer = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(-tf.reduce_sum(y * tf.log(tf.nn.softmax(out_layer)), reduction_indices=[1]))
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(out_layer, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [11]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(epoch):
       _, c=sess.run([train_op,loss_op], feed_dict={X: X_train, y:[t for t in y_train.as_matrix()]})

    # Calculate accuracy 
    print("Testing Accuracy:")
    print(sess.run(accuracy,feed_dict={X: X_test, y:[t for t in y_test.as_matrix()]}))

Testing Accuracy:
0.98
